<a href="https://colab.research.google.com/github/akshayg03/ActiveLearning/blob/master/MLAss2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>